In [2]:
import re 

from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders  import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA



from langchain_community.vectorstores import FAISS

### Langgraph docs loader


In [3]:
from langchain_community.document_loaders  import PyPDFLoader


loader = PyPDFLoader("./data/PERMENPAN NOMOR 38 TAHUN 2017.pdf")
documents = loader.load()

In [4]:
documents[0]

Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA DAN\nREFORMASI BIROKRASI REPUBLIK INDONESIA\nNOMOR 38 TAHUN 2017\nTENTANG\nSTANDAR KOMPETENSI JABATAN APARATUR SIPIL NEGARA\nDENGAN RAHMAT TUHAN YANG MAHA ESA\nMENTERI PENDAYAGUNAAN APARATUR NEGARA\nDAN REFORMASI BIROKRASI REPUBLIK INDONESIA,\nMenimbang : bahwa untuk melaksanakan ketentuan Pasal 55 ayat (5),\nPasal 109 ayat (4) dan ayat (5) dan Pasal 166 ayat (2)\nPeraturan Pemerintah Nomor 11 Tahun 2017 tentang\nManajemen Pegawai Negeri Sipil, perlu menetapkan Peraturan\nMenteri Pendayagunaan Aparatur Negara dan Reformasi\nBirokrasi tentang Standar Kompetensi Jabatan Aparatur Sipil\nNegara;\nMengingat : 1. Undang-Undang Nomor 5 Tahun 2014 tentang Aparatur\nSipil Negara (Lembaran Negara Republik Indonesia Tahun\n2014 Nomor 6, Tambahan Lembaran

In [5]:
import pprint

pprint.pp(documents[0].metadata)

{'producer': 'PyPDF',
 'creator': 'PyPDF',
 'creationdate': '',
 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf',
 'total_pages': 108,
 'page': 0,
 'page_label': '1'}


### Preprocessing

In [6]:
import re 
def preprocess(text): 
    # Menghilangkan footer nomor halaman seperti "- 5 -"
    text = re.sub(r'-\s*\d+\s*-', '', text)
    text = re.sub(r'\n\s*\n', '\n\n', text)
    text = re.sub(r'\n(?!\n)', ' ', text)
    text = re.sub(r' +', ' ', text)

    return text.strip().lower()

for doc in documents:
    doc.page_content = preprocess(doc.page_content) 



In [7]:
documents[0:5]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

In [8]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    
    chunk_overlap=200,  
    length_function=len
)



docs_split = text_splitter.split_documents(documents)

In [9]:
docs_split[:10]

[Document(metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 0, 'page_label': '1'}, page_content='peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia nomor 38 tahun 2017 tentang standar kompetensi jabatan aparatur sipil negara dengan rahmat tuhan yang maha esa menteri pendayagunaan aparatur negara dan reformasi birokrasi republik indonesia, menimbang : bahwa untuk melaksanakan ketentuan pasal 55 ayat (5), pasal 109 ayat (4) dan ayat (5) dan pasal 166 ayat (2) peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai negeri sipil, perlu menetapkan peraturan menteri pendayagunaan aparatur negara dan reformasi birokrasi tentang standar kompetensi jabatan aparatur sipil negara; mengingat : 1. undang-undang nomor 5 tahun 2014 tentang aparatur sipil negara (lembaran negara republik indonesia tahun 2014 nomor 6, tambahan lembaran negara republik

### Configuration

In [10]:
BASE_URL = "https://vllm.emyulabs.my.id/v1"
API_KEY = "sk-local-test"

client = OpenAI(base_url=BASE_URL,
                api_key=API_KEY, 
                timeout=30.0,
                max_retries=3)

### Embedding model

In [11]:

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = OpenAIEmbeddings(
    base_url=BASE_URL,
    api_key=API_KEY,
    model="Qwen/Qwen3-Embedding-0.6B" 
)


db = FAISS.from_documents(docs_split, embedding_model)

### Instruct model

In [12]:
from langchain_openai import ChatOpenAI
llm_chat = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY, 
    model="Qwen/Qwen2.5-coder-1.5B-Instruct", 
    temperature=0.7,
    max_tokens=256,
    streaming=True,
    verbose=True,
)

### Retrieval - Generation

In [13]:
retriever = db.as_retriever(
    search_type = "similarity", # option = mmr or similarity_with_threshold
    search_kwargs = {"k": 10 }
)

# retriever_mmr = db.as_retriever(
#     # Ganti search_type menjadi "mmr"
#     search_type="mmr",
#     search_kwargs={
#         "k": 10,           
#         "fetch_k": 20,   
#         "lambda_mult": 0.5 
#     }
# )

query = """Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?"""

retrieved_docs = retriever.get_relevant_documents(query)

print(f"Dokumen terambil dari k :{retrieved_docs}")

C:\Users\erwin\AppData\Local\Temp\ipykernel_36152\4219862953.py:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


Dokumen terambil dari k :[Document(id='1facc327-cf6b-4d20-8d3d-742c9b47d021', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 73, 'page_label': '74'}, page_content='pemahaman yang sama atas instruksi yang diterima/diberikan 1.3.mampu melaksanakan kegiatan surat menyurat sesuai tata naskah organisasi. 4. orientasi pada hasil 1 bertanggung jawab untuk 1.1.menyelesaikan tugas dengan tuntas; dapat diandalkan;'), Document(id='78d324cd-6388-41dc-8fb3-a146528832c6', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 45, 'page_label': '46'}, page_content='dan pemahaman lintas organisasi untuk secara efektif memfasilitasi kebutuhan kelompok yang lebih besar dengan cara-cara yang mengikuti standar objektif, transparan, profesional, sehingga tidak merugikan para pihak di lingkup pelayanan publ

In [ ]:
#Prompt
from langchain.prompts import PromptTemplate

# Template ini akan diisi dengan dokumen dari retriever dan pertanyaan dari query
prompt_template = """
Gunakan potongan-potongan konteks berikut untuk menjawab pertanyaan pengguna.
Jawablah secara ringkas dan jelas dalam Bahasa Indonesia.
Jika Anda tidak tahu jawabannya berdasarkan konteks yang diberikan, katakan saja bahwa Anda tidak dapat menemukan informasinya, jangan mencoba mengarang jawaban.

KONTEKS:
{context}

PERTANYAAN:
{question}

JAWABAN YANG MEMBANTU:
"""

CUSTOM_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [15]:
from langchain.chains import RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
    llm = llm_chat,
    chain_type = "stuff", 
    retriever = retriever,
    chain_type_kwargs = {"prompt": CUSTOM_PROMPT},
    return_source_documents=True
)

In [16]:
# Menjalankan query melalui RAG Chain
response = rag_chain.invoke(query)

print("\nPERTANYAAN:")
print(query)

print("\nJAWABAN YANG DIHASILKAN LLM:")
print(response['result'])

print("\nDOCUMENT:")
print(response['source_documents'])


retrieved_docs


PERTANYAAN:
Apa inti dalam PERATURAN MENTERI PENDAYAGUNAAN APARATUR NEGARA
 DAN REFORMASI BIROKRASI TENTANG STANDAR
 KOMPETENSI JABATAN APARATUR SIPIL NEGARA ?

JAWABAN YANG DIHASILKAN LLM:
Peraturan Menteri Pendayagunaan Aparatur Negara dan Reformasi Birokrasi Tentang Standar Kompetensi Jabatan Aparatur Sipil Negara adalah peraturan yang bertujuan untuk mengatur dan mengajarkan standar kompetensi jabatan dalam bidang Aparatur Sipil Negara. Ini mencakup beberapa kompetensi penting yang dilakukan oleh pegawai di bidang tersebut, termasuk:

1. Mengerti dan menerapkan norma-norma secara konsisten dalam setiap situasi, pada unit kerja terkecil/kelompok kerjanya.
2. Memberikan informasi yang dapat dipercaya sesuai dengan etika organisasi.
3. Membangun komitmen yang tinggi untuk menyelesaikan tugas tim.
4. Menyampaikan informasi dengan jelas, lengkap, pemahaman yang sama.
5. Melaksanakan kegiatan surat menyurat sesuai tata naskah organisasi.
6. Menyelesaikan tugas dengan tuntas, baik secara

[Document(id='1facc327-cf6b-4d20-8d3d-742c9b47d021', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 73, 'page_label': '74'}, page_content='pemahaman yang sama atas instruksi yang diterima/diberikan 1.3.mampu melaksanakan kegiatan surat menyurat sesuai tata naskah organisasi. 4. orientasi pada hasil 1 bertanggung jawab untuk 1.1.menyelesaikan tugas dengan tuntas; dapat diandalkan;'),
 Document(id='78d324cd-6388-41dc-8fb3-a146528832c6', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '', 'source': './data/PERMENPAN NOMOR 38 TAHUN 2017.pdf', 'total_pages': 108, 'page': 45, 'page_label': '46'}, page_content='dan pemahaman lintas organisasi untuk secara efektif memfasilitasi kebutuhan kelompok yang lebih besar dengan cara-cara yang mengikuti standar objektif, transparan, profesional, sehingga tidak merugikan para pihak di lingkup pelayanan publik unit kerjanya; 3.3. m

### Rag untuk penilaian


In [ ]:
llm_assessor = ChatOpenAI(
    base_url=BASE_URL,
    api_key=API_KEY,
    model="Qwen/Qwen2.5-coder-1.5B-Instruct",
    temperature=0.3, 
    max_tokens=512,
    streaming=False,
    verbose=False,
)

In [ ]:
from prompt.prompt import MANAGERIAL_ASSESSMENT_PROMPT
from langchain.chains import LLMChain


# LLM CHAIN 
assesment_chain =  LLMChain(
    llm = llm_assessor, 
    prompt= MANAGERIAL_ASSESSMENT_PROMPT
)


def competency_asessment(nama, jabatan, jawaban_test, kompetensi, level_target) : 
    query = f"kompetensi {kompetensi} level {level_target} indikator perilaku"
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])

    assesment_chain_result = assesment_chain.invoke({
        "context": context,
        "nama": nama,
        "jabatan": jabatan,
        "jawaban": jawaban_test,
        "kompetensi": kompetensi,
        "level_target": level_target
    })

    return {
        "hasil": assesment_chain_result['text'],
        "sumber": relevant_docs  
    }
    


In [ ]:
test_cases = [
    {
        "nama": "Budi Santoso",
        "jabatan": "Kepala Seksi",
        "jawaban_test": """
        Dalam menyelesaikan konflik tim, saya pertama-tama mengumpulkan semua pihak 
        untuk mendengarkan perspektif masing-masing. Saya menggunakan pendekatan win-win solution 
        dengan mencari titik temu yang menguntungkan semua pihak. Setelah mencapai kesepakatan, 
        saya memastikan implementasinya dengan monitoring berkala dan evaluasi hasil.
        """,
        "kompetensi": "Kerjasama",
        "level_target": 3
    },
    {
        "nama": "Siti Rahayu",
        "jabatan": "Analis Kepegawaian",
        "kompetensi": "Manajemen SDM", 
        "jawaban_test": """
        Untuk menyusun perencanaan SDM, saya melakukan analisis kebutuhan berdasarkan 
        rencana strategis organisasi. Saya menggunakan data historis dan proyeksi kebutuhan 
        untuk menentukan jumlah dan kualifikasi pegawai yang dibutuhkan. 
        Saya juga mempertimbangkan aspek pengembangan karir dan suksesi planning.
        """,
        "level_target": 2
    },
    {
        "nama": "Ani Suryani",
        "jabatan": "Inspektur Madya",
        "jawaban_test": """
        Ketika menemukan adanya potensi penyalahgunaan wewenang di laporan, saya langsung
        melakukan verifikasi silang terhadap data dan peraturan yang berlaku tanpa memandang
        siapa yang terlibat. Saya memastikan semua temuan didukung oleh bukti yang kuat
        dan melaporkannya sesuai prosedur meskipun ada tekanan dari berbagai pihak.
        """,
        "kompetensi": "Integritas",
        "level_target": 4
    }
]


for i, person in enumerate (test_cases, 1): 
    print(f"Penilaian ke {i} dengan nama {person["nama"]}")
    print(f"Jabatan {person["jabatan"]}")
    print("="*60)

    hasil_penilaian = competency_asessment(
        nama=person["nama"],
        jabatan=person["jabatan"],
        jawaban_test=person["jawaban_test"],
        kompetensi=person["kompetensi"],
        level_target=person["level_target"]
    )

    print("Hasil Penilaian")
    print(hasil_penilaian["hasil"])

    print("\n DOKUMEN SUMBER:")
    print("-" * 30)
    for doc_num, doc in enumerate(hasil_penilaian['sumber'], 1):
        page_label = doc.metadata.get('page_label', 'N/A')
        print(f"  Sumber #{doc_num} (Halaman: {page_label}):")
        print(f"  \"{doc.page_content[:250].strip()}...\"\n") 
        

Penilaian ke 1 dengan nama Budi Santoso
Jabatan Kepala Seksi
Hasil Penilaian
### HASIL PENILAIAN

#### SKOR: 3
#### JUSTIFIKASI:
Budi Santoso berhasil menyelesaikan konflik tim dengan baik. Dia mengumpulkan semua pihak untuk mendengarkan perspektif masing-masing dan mencari titik temu yang menguntungkan semua pihak. Namun, ia juga melakukan monitoring berkala dan evaluasi hasil, yang membuat penyelesaian menjadi lebih efektif. 

#### REKOMENDASI:
Berdasarkan penilaian, Budi Santoso telah mencapai level 3 dalam kompetensi Kerjasama. Namun, ia bisa meningkatkan pengetahuan tentang cara mengimplementasikan penyelesaian konflik tim dengan lebih efektif. Misalnya, ia bisa belajar lebih banyak tentang metode monitoring dan evaluasi yang efektif, atau belajar bagaimana mengidentifikasi dan menganalisis masalah konflik tim.

 DOKUMEN SUMBER:
------------------------------
  Sumber #1 (Halaman: 1):
  "indonesia nomor 5494); 2. peraturan pemerintah nomor 11 tahun 2017 tentang manajemen pegawai n